In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd

In [26]:
df = pd.read_excel('file:///H:\lvpcprojects\evoLVe\GIS\TrafficX\TrafficX_Updated_Nov16.xlsx', 
                   sheet_name = 'TrafficData for Stack').iloc[:,:6]
df1 = (df.assign(cumID = df.groupby(['Route', 'ID']).cumcount())
.set_index(['Route', 'ID', 'cumID']) .unstack(-1).sort_index(1, 1))

#Combine the level labels to format the column names into one row
df1.columns = [f'{x}{y}' for x,y in df1.columns]

#Format the dataframe by resetting index
df2 = df1.reset_index()

#Replace the empty values with 'NA'.
df2[['Truck_%0', 'Truck_%1', 'Truck_%2', 'Truck_%3', 'Truck_%4']] = (df2
[['Truck_%0', 'Truck_%1', 'Truck_%2', 'Truck_%3', 'Truck_%4']].fillna('NA'))

#Separate the original points into different groups based on number of records of each one.
dff = pd.read_excel('file:///H:\lvpcprojects\evoLVe\GIS\TrafficX\TrafficX_Updated_Nov16.xlsx', 
                   sheet_name = 'TrafficData for Stack').iloc[:,:6]
dff1 = dff.groupby(['Route', 'ID']).size().reset_index()
dff1.columns = ['Route', 'ID', 'Count']

#Create each dataframe variable using loop, store them in a dictionary
d = {}
for i in range(1, 6):
    d["dfc{0}".format(i)]= dff1[dff1['Count'] == i]
#Merge each dataframe from last time with df2 that we got previously
l = []
for key in d.keys():
    l.append(pd.merge(d[key], df2, on = ['Route','ID'], how = 'left'))
#Save every result to excel
writer = pd.ExcelWriter('H:\lvpcprojects\evoLVe\GIS\TrafficX\TrafficX_Reorganize.xlsx')
for i in range(len(l)):
    l[i].to_excel(writer, 'Count' + str(i))
writer.save()

In [20]:
#Try to create different variables using for loop
d = {}
for i in range(1, 6):
    d["dfc{0}".format(i)]= dff1[dff1['Count'] == i]
l = []
for key in d.keys():
    l.append(pd.merge(d[key], df2, on = ['Route','ID'], how = 'left'))
len(l)

5

,Route,ID,Count,Truck_%0,Truck_Vol0,Vol0,Year0,Truck_%1,Truck_Vol1,Vol1,...,Vol2,Year2,Truck_%3,Truck_Vol3,Vol3,Year3,Truck_%4,Truck_Vol4,Vol4,Year4
0,22,15,4,NA,NaN,50805.0,2017.0,7.1,3521.0,49640.0,...,51569.0,2010.0,NA,NaN,51034.0,2009.0,NA,NaN,NaN,NaN
1,248,21,4,8.4,928.0,11112.0,2018.0,NA,NaN,12348.0,...,8920.0,2012.0,NA,NaN,9343.0,2009.0,NA,NaN,NaN,NaN
2,33,32,4,NA,NaN,83572.0,2017.0,15,10832.0,72306.0,...,64024.0,2010.0,NA,NaN,62341.0,2009.0,NA,NaN,NaN,NaN
3,33,34,4,NA,NaN,59081.0,2017.0,17.5,9317.0,53274.0,...,44394.0,2010.0,NA,NaN,42988.0,2009.0,NA,NaN,NaN,NaN
4,Tilghman Street,56,4,NA,NaN,18087.0,2018.0,13.5,2672.0,19795.0,...,14934.0,2012.0,NA,NaN,14467.0,2009.0,NA,NaN,NaN,NaN
